## Прогнозирование риска пожара на основе спутниковых снимков с помощью модели ResNet50
- Как были получены данные (инфо из kaggle):
    - Используя координаты широты и долготы для каждого реального очага пожара (>0,01 акра сгоревшей площади) \
    были извлечены спутниковые снимки этих областей с помощью MapBox API. \
    Данные предназначены для разработки модели, способной предсказывать риск возникновения пожара в определённой области.
- Данная модель после обучения дожна принимать на вход спутниковый снимок и делать предсказание возможен ли пожар на данной территории или нет.

#### Датасет

- https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset

In [39]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import os
from PIL import ImageFile

In [41]:
# импорт датасета из kaggle --- рекомендуется взять данные так, если запускать в colab ----

# import kagglehub
# path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")
# print("Path to dataset files:", path)

In [50]:
# --- для запуска локально рекомендуется сначала скачать датасет ---
# путь к датасету  сохраненному локально, необходимо задать!!!
# path="wildfire_data_set_path"
path="K:\\EducationCV\\first_year\\hackathon\\WildFire"

In [62]:
# Посмотреть, что данные видны
print(os.listdir(path))
print(os.listdir(path + "/train/wildfire"))

['.venv', 'archive.zip', 'example.txt', 'main.py', 'test', 'train', 'valid']
['-57.11902,51.47242.jpg', '-57.8088,51.44634.jpg', '-58.657,51.1945.jpg', '-58.71402,51.32554.jpg', '-58.7312,51.2023.jpg', '-58.81589,51.78597.jpg', '-58.88032,51.79047.jpg', '-58.96863,50.90078.jpg', '-59.0008,50.8606.jpg', '-59.08647,51.09772.jpg', '-59.12153,51.85762.jpg', '-59.13463,51.86481.jpg', '-59.1904,50.7664.jpg', '-59.2239,51.9222.jpg', '-59.23837,51.77878.jpg', '-59.2722,50.7361.jpg', '-59.2879,51.8789.jpg', '-59.3422,52.0486.jpg', '-60.0594,50.2056.jpg', '-60.1853,50.2269.jpg', '-60.2869,50.2306.jpg', '-60.5963,50.4126.jpg', '-60.6811,50.3211.jpg', '-60.8066,50.26709.jpg', '-60.85208,51.20354.jpg', '-60.8867,51.1911.jpg', '-60.9444,50.3885.jpg', '-60.9831,50.4017.jpg', '-60.9878,50.4112.jpg', '-61.33042,51.79317.jpg', '-61.35796,51.09233.jpg', '-61.43161,50.86211.jpg', '-61.4492,51.7519.jpg', '-61.47189,47.78507.jpg', '-61.56041,51.11211.jpg', '-61.5607,50.52878.jpg', '-61.59782,50.20084.jpg', 

In [16]:
# Список преобразований, которые применяеются к каждому изображению перед тем, как передать его в нейросеть

 # Размер входного изображения ResNet-50 составляет 224x224x3.
 # Входное изображение должно быть квадратным, с шириной и высотой 224 пикселя и 3 каналами (RGB)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # размер входных изображений для ResNet50
    transforms.ToTensor(), # конвертация изображение в тензор PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация под предобученные веса
                                                          # ResNet50 была обучена на ImageNet,
                                                          # где средние значения [0.485, 0.456, 0.406], а std [0.229, 0.224, 0.225]
])

In [18]:
# # проверяем, доступна ли в системе GPU (CUDA). Если да, то модель и данные будут загружены на GPU, если нет — будут использовать CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [20]:
# Загрузка данных и подготовка их к обучению модели
train_path = path + "/train"
val_path = path + "/valid"

# читает изображения и назначает им метки wildfire → label = 0, nowildfire → label = 1
train_data = datasets.ImageFolder(root=train_path, transform=transform) # тренировочные данные
val_data = datasets.ImageFolder(root=val_path, transform=transform) # валидационные данные

# делает мини-батчи и управляет их подачей в модель
train_loader = DataLoader(train_data, batch_size=32, shuffle=True) # перемешивает данные перед каждой эпохой (важно для обучения)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False) # порядок остается фиксированным (не влияет на валидацию)

print(f"Classes: {train_data.classes}")  # Должно вывести ['nowildfire', 'wildfire']

# определение количества классов
# нужно, чтобы настроить последний слой модели
num_classes = len(train_data.classes)

Classes: ['nowildfire', 'wildfire']


In [22]:
# загружаем предобученную модель ResNet50 с весами ImageNet
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# отключаем обновление весов во всей сети ResNet50, кроме последнего слоя
for param in model.parameters():
    param.requires_grad = False  # заморозка веса предобученной части

# замена последнего слоя (fc – fully connected) на новый с 2 выходами (для wildfire и nowildfire).
model.fc = nn.Linear(model.fc.in_features, num_classes)

# перемещаем модель на GPU или CPU
# модель и данные должны быть на одном устройстве, иначе будут ошибки!
model = model.to(device)

In [24]:
# определяем функцию потерь (Cross Entropy Loss), которая используется для задачи классификации
# чем меньше loss → тем точнее предсказание
criterion = nn.CrossEntropyLoss() # вычисляет разницу между предсказанными вероятностями и истинными метками классов

# оптимизатор Adam, который обновляет веса модели.
# обновляются только параметры последнего слоя fc
# lr=0.001 — скорость обучения (learning rate)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [26]:
# в датасете присутствют битые картинки
# поэтому поставлен флаг, что загружать битые изображения и не выбрасывать исключения
ImageFile.LOAD_TRUNCATED_IMAGES = True

# количество эпох
num_epochs = 10

# действия в каждой эпохе
for epoch in range(num_epochs):
    # устанавливаем модель в режим обучения
    model.train()

    # сумма потерь (loss) в текущей эпохе
    running_loss = 0.0

    # train_loader содержит батчи (группы изображений и группы меток)
    for images, labels in train_loader:

        # перемещаем images и labels на GPU или CPU
        images, labels = images.to(device), labels.to(device)
        # обнуление градиентов каждый раз перед loss.backward()
        optimizer.zero_grad()
        # пропускаем images через модель
        outputs = model(images)
        # вычисление разницы между предсказанными значениями (outputs) и истинными метками (labels).
        loss = criterion(outputs, labels)
        # вычисление градиента ошибки по отношению к параметрам модели с помощью обратного распространения ошибки
        loss.backward()
        # обновляем веса модели с учётом градиентов
        optimizer.step()
        # преобразует loss в число
        # суммирует все ошибки за эпоху
        running_loss += loss.item()

    # делим running_loss на количество батчей (len(train_loader)) → получаем средний loss
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Training Finished!")

Epoch 1, Loss: 0.1621457986468666
Epoch 2, Loss: 0.11421618675498044
Epoch 3, Loss: 0.10335398027693053
Epoch 4, Loss: 0.09363709122580466
Epoch 5, Loss: 0.09207087480581226
Epoch 6, Loss: 0.08920500690194268
Epoch 7, Loss: 0.08684630563628346
Epoch 8, Loss: 0.08452659662213396
Epoch 9, Loss: 0.08573174322034817
Epoch 10, Loss: 0.08273452035757865
Training Finished!


In [28]:
# оценим точность модели на валидационном датасете

# переведем модель в режим оценки
model.eval()

correct = 0
total = 0

# отключаем вычисление градиентов
with torch.no_grad():
    # проходим по валидационным изображениям и их истинным меткам
    for images, labels in val_loader:
        # перемещаем изображения и метки на тот же девайс, что и модель
        images, labels = images.to(device), labels.to(device)
        # пропускаем изображения через модель
        outputs = model(images)
        # predicted содержит предсказанные классы (0 или 1 для каждого изображения)
        _, predicted = torch.max(outputs, 1)
        # считаем общее число изображений
        total += labels.size(0)
        # создаем булевый тензор и считаем количество верных предсказаний, переводим в int
        correct += (predicted == labels).sum().item()

# выводим точность модели
print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Validation Accuracy: 97.11%


In [35]:
# сохраним только веса модели (без архитектуры)
torch.save(model.state_dict(), "wildfire_prediction_resnet50.pth")
print("The model is saved!")

The model is saved!
